In [142]:
import pandas as pd
import numpy as np
from numpy import where, mean, std
import nltk, string, math
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_validate, train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.datasets import make_classification
from itertools import product
from imblearn.over_sampling import ADASYN, RandomOverSampler, BorderlineSMOTE, SMOTE, SVMSMOTE, SMOTENC, BorderlineSMOTE

from imblearn.combine import SMOTETomek, SMOTEENN

from collections import Counter
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from imblearn.under_sampling import RandomUnderSampler, OneSidedSelection, ClusterCentroids

from sklearn.feature_selection import RFE, SelectKBest, f_classif, VarianceThreshold, SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR, SVC
from sklearn.calibration import LinearSVC
from imblearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    make_scorer
)


# Wczytanie danych
Pierwsza kolumna to "sample_id", a ostatnia kolumna to CLASS oznaczająca klasę. Reszta to wartości liczbowe separowane znakiem ";"

In [143]:
RNG = np.random.RandomState(42)
CLASS = "CLASS"
SAMPLE_ID = "sample_id"

alleu_data = pd.read_csv("csv/ALL_GSE13425_FINAL.csv", sep=';')
btu_data = pd.read_csv("csv/BTu_GSE4290_FINAL.csv", sep=';')
hec_data = pd.read_csv("csv/HeC_GSE14323_FINAL.csv", sep=';')
hff_data = pd.read_csv("csv/HFF_GSE5406_FINAL.csv", sep=';')
sps_data = pd.read_csv("csv/SPs_GSE13355_FINAL.csv", sep=';')
ssh_data = pd.read_csv("csv/SSh_GSE13904_FINAL.csv", sep=';')


In [144]:
def clean_data(df):
    s = df.drop(columns=[CLASS, SAMPLE_ID]).select_dtypes(include='object').columns
    df[s] = df[s].apply(lambda x: x.astype(float))

    return df

datasets = {
    "ALL": clean_data(alleu_data),
    "BTU": clean_data(btu_data),
    "HEC": clean_data(hec_data),
    "HFF": clean_data(hff_data),
    "SPS": clean_data(sps_data),
    "SSH": clean_data(ssh_data)
}

all_results = {}

def add_results(method_label, dict_obj):
    if method_label not in all_results:
        all_results[method_label] = {}
    for dataset_name, result in dict_obj.items():
        all_results[method_label][dataset_name] = result

In [145]:
def show_data(df, name):
    print(f"{name} Data:")
    print(df.describe())
    print(f"Liczba właściwości: {df.shape[1] - 1}")
    print(f"Liczba klas: {df[CLASS].nunique()}")
    class_counts = df[CLASS].value_counts().sort_index()
    for label, count in class_counts.items():
        print(f"\tKlasa {label} ({label}): {count}")
    
    print(f"Brakujące wartości: {df.isnull().sum().sum()}\n")

# show_data(datasets["ALL"], "LEUKEMIA")

Wykres danych czystych

In [146]:
def plot_datasets(datasets):
    n_plots = len(datasets)
    cols = 2
    rows = (n_plots + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(10, 4 * rows))
    axes = axes.flatten()
    for idx, (name, data) in enumerate(datasets.items()):
        print(f"Tworzenie wykresu {name}...")
        X = data.drop(columns=[CLASS, SAMPLE_ID]).values
        y = data[CLASS].values
        
        # summarize class distribution
        counter = Counter(y)
        ax = axes[idx]
        print(counter)
        # scatter plot of examples by class label
        for label, _ in counter.items():
            row_ix = where(y == label)[0]
            ax.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
        ax.set_title(name)
        ax.legend()

    for j in range(idx + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

#plot_datasets(datasets)

Modele, oversamplery i undersamplery

In [147]:
models = {
    "DecisionTree": DecisionTreeClassifier(random_state=RNG),
    "AdaBoost": AdaBoostClassifier(n_estimators=25, random_state=RNG),
    "RandomForest": RandomForestClassifier(random_state=RNG, n_jobs=-1),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=RNG, max_iter=10, max_depth=3, learning_rate=0.3)
}
oversamplers = {
    "OverSampler": RandomOverSampler(random_state=RNG),
    "SMOTE": SMOTE(k_neighbors=1, random_state=RNG),
    "BorderlineSmote": BorderlineSMOTE(k_neighbors=1, random_state=RNG),
    "SVMSMOTE": SVMSMOTE(k_neighbors=1, random_state=RNG)
}

undersamplers = {
    "UnderSampler": RandomUnderSampler(random_state=RNG),
    "OneSidedSelection": OneSidedSelection(random_state=RNG, n_neighbors=1, n_jobs=-1),
    "ClusterCentroids": ClusterCentroids(random_state=RNG)
}

hybrids = {
    "SMOTETomek": SMOTETomek(random_state=RNG, smote=oversamplers["SMOTE"], n_jobs=-1),
    "SMOTEENN": SMOTEENN(random_state=RNG, smote=oversamplers["SMOTE"], n_jobs=-1)
}

# estimator = LinearSVC(C=0.01, penalty="l1", dual=False, max_iter=4000,  random_state=RNG)
estimator = SVC(kernel="linear", C=0.0000001, tol=0.0001, max_iter=4000, random_state=RNG)

excluded_from_ALL = ["SMOTE", "SVMSMOTE"]

Funkcje pomocnicze

In [ ]:
def evaluate_model(name, label, pipeline, X, y, cv, comments=False):
    scoring = {
        "accuracy": make_scorer(accuracy_score),
        "balanced_accuracy": make_scorer(balanced_accuracy_score),
        "precision": make_scorer(precision_score, average="macro", zero_division=0),
        "recall": make_scorer(recall_score, average="macro", zero_division=0),
        "f1_micro": make_scorer(f1_score, average="macro"),
    }
    
    metrics = cross_validate(pipeline, X, y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score=False)
    accuracy = metrics["test_accuracy"].mean()
    bal_accuracy = metrics["test_balanced_accuracy"].mean()
    precision = metrics["test_precision"].mean()
    recall = metrics["test_recall"].mean()
    f1_sc = metrics["test_f1_micro"].mean()
    st_dev = metrics["test_f1_micro"].std()
    
    
    if comments:
        print(f"\n=== Wyniki ({name} | {label}) ===")
        print(f"Accuracy:    \t{accuracy:.3f}")
        # print(f"Balanced acc:\t{bal_accuracy:.3f}")
        # print(f"Precision:   \t{precision:.3f}")
        # print(f"Recall:      \t{recall:.3f}")
        # print(f"F1 Score:    \t{f1_sc:.3}")
        # print(f"Std dev:     \t{st_dev:.3f}")
    

    return {
        "dataset": name,
        "label": label,
        "f1_score": f1_sc,
        "accuracy": accuracy,
        "balanced_accuracy": bal_accuracy,
        "precision": precision,
        "recall": recall,
        "stdev": st_dev
    }
    
def plot_resampled_data(steps, X, y, ax, title, sel_features=False):
    resample_steps = steps.copy()
    resample_steps.pop()
    if sel_features: resample_steps.pop()
    resample_steps = resample_steps[1:]
    res_pipeline = Pipeline(resample_steps)

    X_res, y_res = res_pipeline.fit_resample(X, y)
    counter = Counter(y_res)
    print("Rozkład po resamplingu:", counter)

    for label_ in counter:
        idx = np.where(y_res == label_)[0]
        ax.scatter(X_res[idx, 0], X_res[idx, 1], label=str(label_))

    ax.set_title(title)
    ax.legend()

def create_empty_result(name, label):
    return {
        "dataset": name,
        "label": label,
        "f1_score": np.nan,
        "accuracy": np.nan,
        "balanced_accuracy": np.nan,
        "precision": np.nan,
        "recall": np.nan,
        "stdev": np.nan
    }


Funkcja ogólna klasyfikacji


In [149]:
def run_classification(
        classifier, 
        oversampler=None, 
        undersampler=None, 
        plot_resampled=False, 
        label="",
        skip_ALL=False,
        comments=False,
        sel_features=False
    ):
    
    if plot_resampled:
        n_datasets = len(datasets)
        cols = 3
        rows = (n_datasets + cols - 1) // cols
        fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
        axes = axes.flatten()
        plot_index = 0
    
    res = {}
    for name, data in datasets.items():
        if skip_ALL and name == "ALL":
            print(f"\nPomijanie wykluczonego zbioru {name}.\n")
            res[name] = create_empty_result(name, label)
            continue
        
        X = data.drop(columns=[CLASS, SAMPLE_ID]).values
        y = data[CLASS].values

        steps = [("scaler", MinMaxScaler())]
        if undersampler: steps.append(("under", undersampler))
        if oversampler: steps.append(("over", oversampler))
        
        if sel_features:
            # print("Selekcja cech...")
            n_select = max(1, int(0.1 * X.shape[1]))
            selector = SelectFromModel(estimator=estimator, max_features=n_select, threshold=-np.inf)
            steps.append(("selector", selector))
        
        steps.append(("model", classifier))
        pipeline = Pipeline(steps=steps)

        cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=3, random_state=RNG)
        result = evaluate_model(name, label, pipeline, X, y, cv, comments=comments)
        res[name] = result
        
        if plot_resampled and (oversampler or undersampler):
            ax = axes[plot_index]
            plot_index += 1
            steps = pipeline.steps
            plot_resampled_data(steps, X, y, ax, f"{name} - {label}", sel_features=sel_features)
    
    if plot_resampled:
        plt.tight_layout()
        plt.show()
    
    return res

POTĘŻNA PĘTLA

In [150]:
sel_features = False



for md_name, model in models.items():
    label = md_name
    print(f"Klasyfikacja {label}...")
    res = run_classification(classifier=model, label=label, oversampler=None, undersampler=None, sel_features=sel_features)
    add_results(label, res)
    
    
    for h_name, h in hybrids.items():
        label = f"{md_name}_{h_name}"
        print(f"Klasyfikacja {label}...")
        res = run_classification(classifier=model, label=label, oversampler=h, undersampler=None, skip_ALL=(h_name in excluded_from_ALL), sel_features=sel_features)
        add_results(label, res)
    
    for ove_name, ove in oversamplers.items():
        label = f"{md_name}_{ove_name}"
        print(f"Klasyfikacja {label}...")
        res = run_classification(classifier=model, label=label, oversampler=ove, undersampler=None, skip_ALL=(ove_name in excluded_from_ALL), sel_features=sel_features)
        add_results(label, res)

    for ove_name, und_name in product(oversamplers.keys(), undersamplers.keys()):
        label = f"{md_name}_{ove_name}_{und_name}"
        print(f"Klasyfikacja {label}...")
        
        # Pomijanie kombinacji undersamplerów z SVMSMOTE ze względu na generowanie błędów        
        if ove_name == "SVMSMOTE":
            continue
        
        # Pomijanie tej kombinacji bo też błędy wywala    
        if ove_name == "SMOTE" and und_name == "OneSidedSelection":
            continue
        
        res = run_classification(
            classifier=model, label=label, oversampler=oversamplers[ove_name], undersampler=undersamplers[und_name], 
            skip_ALL=(ove_name in excluded_from_ALL), sel_features=sel_features
        )
        add_results(label, res)


Klasyfikacja DecisionTree...
Klasyfikacja DecisionTree_SMOTETomek...
Klasyfikacja DecisionTree_SMOTEENN...
Klasyfikacja DecisionTree_OverSampler...
Klasyfikacja DecisionTree_SMOTE...

Pomijanie wykluczonego zbioru ALL.



AttributeError: `np.NaN` was removed in the NumPy 2.0 release. Use `np.nan` instead.

Klasyfikacja na danych czystych (Drzewo decyzyjne)

In [ ]:
import time


label="clean"
# start_time = time.time()
# res = run_classification(
#         classifier=models["DecisionTree"],
#         label=label,
#         comments=True,
#         sel_features=True 
# )
# print("%s seconds" % (time.time() - start_time))

# for md_name, model in models.items():
#         start_time = time.time()
#         print("Klasyfikacja: ", md_name)
#         res = run_classification(
#         classifier=model,
#         label=label,
#         comments=True,
#         sel_features=True 
#         )
#         print("%s seconds" % (time.time() - start_time))

# # add_results(label, res)


# Wyniki czyste (DecisionTree)
# # === Wyniki (ALL | clean) ===
# # Accuracy:    	0.702

# # === Wyniki (BTU | clean) ===
# # Accuracy:    	0.591

# # === Wyniki (HEC | clean) ===
# # Accuracy:    	0.710

# # === Wyniki (HFF | clean) ===
# # Accuracy:    	0.517

# # === Wyniki (SPS | clean) ===
# # Accuracy:    	0.698

# # === Wyniki (SSH | clean) ===
# # Accuracy:    	0.395
# # --- 10.617483854293823 seconds ---


Klasyfikacja (Testy)

In [ ]:
# Klasyfikacja:  DecisionTree
# Selekcja cech...

# === Wyniki (ALL | clean) ===
# Accuracy:    	0.698
# Selekcja cech...

# === Wyniki (BTU | clean) ===
# Accuracy:    	0.611
# Selekcja cech...

# === Wyniki (HEC | clean) ===
# Accuracy:    	0.664
# Selekcja cech...

# === Wyniki (HFF | clean) ===
# Accuracy:    	0.552
# Selekcja cech...

# === Wyniki (SPS | clean) ===
# Accuracy:    	0.737
# Selekcja cech...

# === Wyniki (SSH | clean) ===
# Accuracy:    	0.395
# 11.335190773010254 seconds
# Klasyfikacja:  AdaBoost
# Selekcja cech...

# === Wyniki (ALL | clean) ===
# Accuracy:    	0.737
# Selekcja cech...

# === Wyniki (BTU | clean) ===
# Accuracy:    	0.635
# Selekcja cech...

# === Wyniki (HEC | clean) ===
# Accuracy:    	0.659
# Selekcja cech...

# === Wyniki (HFF | clean) ===
# Accuracy:    	0.537
# Selekcja cech...

# === Wyniki (SPS | clean) ===
# Accuracy:    	0.780
# Selekcja cech...

# === Wyniki (SSH | clean) ===
# Accuracy:    	0.460
# 12.024312734603882 seconds
# Klasyfikacja:  RandomForest
# Selekcja cech...

# === Wyniki (ALL | clean) ===
# Accuracy:    	0.849
# Selekcja cech...

# === Wyniki (BTU | clean) ===
# Accuracy:    	0.696
# Selekcja cech...

# === Wyniki (HEC | clean) ===
# Accuracy:    	0.898
# Selekcja cech...

# === Wyniki (HFF | clean) ===
# Accuracy:    	0.573
# Selekcja cech...

# === Wyniki (SPS | clean) ===
# Accuracy:    	0.807
# Selekcja cech...

# === Wyniki (SSH | clean) ===
# Accuracy:    	0.511
# 7.096845388412476 seconds
# Klasyfikacja:  HistGradientBoosting
# Selekcja cech...

# === Wyniki (ALL | clean) ===
# Accuracy:    	0.854
# Selekcja cech...

# === Wyniki (BTU | clean) ===
# Accuracy:    	0.700
# Selekcja cech...

# === Wyniki (HEC | clean) ===
# Accuracy:    	0.876
# Selekcja cech...

# === Wyniki (HFF | clean) ===
# Accuracy:    	0.602
# Selekcja cech...

# === Wyniki (SPS | clean) ===
# Accuracy:    	0.780
# Selekcja cech...

# === Wyniki (SSH | clean) ===
# Accuracy:    	0.490
# 21.38968801498413 seconds

In [ ]:
# cl = GradientBoostingClassifier(random_state=RNG, n_estimators=50, max_depth=3, learning_rate=0.3)

# Ogólnie: Najlepszy klasyfikator, ale WOOOOOOOOOOOOOLNY
# cl = HistGradientBoostingClassifier(random_state=RNG, max_iter=10, max_depth=3, learning_rate=0.3)

# under = ClusterCentroids(random_state=RNG)
# label="GRADIENT"

# res = run_classification(
#         classifier=cl,
#         # oversampler=ADASYN(random_state=RNG, n_neighbors=1),
#         # undersampler=under,
#         label=label
# )

Uwagi:
- SMOTE ciężko jest zastosować na tym zbiorze danych, bo zbiór ALL posiada dwie klasy z 4-5 próbkami, przez co n_neighbours musi być bardzo małe
- Ogólnie dowolne metody kNN kuleją, jeśli używa się podziału na foldy
- ADASYN nie moze dokonać klasyfikacji bo problemy ze zbyt małym zbiorem danych.
- CVStratifiedKFold nie może być użyty w większych wartościach bo zbiór ALL posiada dwie klasy z 4-5 próbkami, a utworzenie foldu z n_splits >= 4 jest dla niej niemożliwe.
- AdaBoost jest ciężki obliczeniowo, ale daje najlepsze wyniki dla zbioru SSH, który ma najgorsze wyniki.
- Jeśli chodzi o wyniki, adaboost  nieźle poradził sobie ze zbiorami o niskim F1 score, ale miał problemy z ALL i HEC.
- Trzeba wypróbować jakiś lepszy sposób na oversampling, albo zamiast KFoldów użyć coś innego.


EXCEL

In [ ]:
thin = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

STYLES = {
    "model_label": {
        "font": Font(bold=True),
        "alignment": Alignment(horizontal="center"),
        "fill": PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid"),
        "border": thin
    },
    "model_value": {
        "font": Font(bold=True),
        "alignment": Alignment(horizontal="left"),
        "border": thin
    },
    "header": {
        "font": Font(bold=True),
        "alignment": Alignment(horizontal="center"),
        "border": thin
    },
    "metric": {
        "font": Font(bold=True),
        "alignment": Alignment(horizontal="left"),
        "fill": PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid"),
        "border": thin
    },
    "value": {
        "alignment": Alignment(horizontal="center"),
        "border": thin
    }
}

def apply_style(cell, style_name):
    style = STYLES[style_name]
    for attr, val in style.items():
        setattr(cell, attr, val)

In [ ]:
def export_to_excel(all_results, file_path="wyniki.xlsx"):
    print(all_results)
    
    wb = Workbook()
    ws = wb.active
    ws.title = "Wyniki"
    
    current_row = 1
    gap = 3
    
    for label, datasets_dict in all_results.items():
        dataset_names = list(datasets_dict.keys())
        metrics = ["accuracy", "balanced_accuracy", "precision", "recall", "f1_score", "stdev"]

        df = pd.DataFrame(index=metrics, columns=dataset_names)
        for ds_name, result in datasets_dict.items():
            df.loc["accuracy", ds_name] = result["accuracy"]
            df.loc["balanced_accuracy", ds_name] = result["balanced_accuracy"]
            df.loc["precision", ds_name] = result["precision"]
            df.loc["recall", ds_name] = result["recall"]
            df.loc["f1_score", ds_name] = result["f1_score"]
            df.loc["stdev", ds_name] = result["stdev"]
            
        c1 = ws.cell(row=current_row, column=1, value="model")
        apply_style(c1, "model_label")

        c2 = ws.cell(row=current_row, column=2, value=label)
        apply_style(c2, "model_value")

        start_row = current_row + 2
        row_idx = start_row

        for row in dataframe_to_rows(df, index=True, header=True):
            col_idx = 1
            for value in row:
                cell = ws.cell(row=row_idx, column=col_idx, value=value)

                if row_idx == start_row: apply_style(cell, "header")
                elif col_idx == 1: apply_style(cell, "metric")
                else: apply_style(cell, "value")

                col_idx += 1
            row_idx += 1

        current_row = row_idx + gap

    wb.save(file_path)
    print(f"\nZapisano do Excela: {file_path}")

if sel_features:
    file_name = "wyniki_sel_features.xlsx"
else: file_name = "wyniki.xlsx"

export_to_excel(all_results, file_path = file_name)

{'DecisionTree': {'ALL': {'dataset': 'ALL', 'label': 'DecisionTree', 'f1_score': np.float64(0.597353215624917), 'accuracy': np.float64(0.7333333333333334), 'balanced_accuracy': np.float64(0.6102178245035388), 'precision': np.float64(0.6174641235566071), 'recall': np.float64(0.6102178245035388), 'stdev': np.float64(0.08989641778448962)}, 'BTU': {'dataset': 'BTU', 'label': 'DecisionTree', 'f1_score': np.float64(0.5353548982635091), 'accuracy': np.float64(0.5907407407407408), 'balanced_accuracy': np.float64(0.5521326302423865), 'precision': np.float64(0.5389297735367382), 'recall': np.float64(0.5521326302423865), 'stdev': np.float64(0.05950695781636275)}, 'HEC': {'dataset': 'HEC', 'label': 'DecisionTree', 'f1_score': np.float64(0.6697554731217853), 'accuracy': np.float64(0.7150537634408601), 'balanced_accuracy': np.float64(0.6660232631700024), 'precision': np.float64(0.7140639131400001), 'recall': np.float64(0.6660232631700024), 'stdev': np.float64(0.05484143411207743)}, 'HFF': {'dataset'

Zapisywanie do .csv

In [ ]:
import csv

def export_to_csv(all_results, file_path="wyniki.csv"):
    rows = []
    all_columns = set()

    for label, datasets_dict in all_results.items():
        for ds_name, result in datasets_dict.items():
            for metric in ["accuracy", "balanced_accuracy", "precision", "recall", "f1_score", "stdev"]:
                col = f"{ds_name}_{metric}"
                all_columns.add(col)

    all_columns = sorted(all_columns)
    header = ["model"] + all_columns

    for label, datasets_dict in all_results.items():
        row = {"model": label}

        for col in all_columns:
            row[col] = ""

        for ds_name, result in datasets_dict.items():
            for metric in ["accuracy", "balanced_accuracy", "precision", "recall", "f1_score", "stdev"]:
                col = f"{ds_name}_{metric}"
                if col in row:
                    row[col] = result[metric]

        rows.append(row)

    # Zapis CSV
    with open(file_path, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=header, delimiter=";")
        writer.writeheader()
        writer.writerows(rows)

    print(f"\nZapisano wyniki do pliku CSV: {file_path}")


if sel_features:
    file_name = "wyniki_sel_features.csv"
else: file_name = "wyniki.csv"
export_to_csv(all_results, file_path=file_name)


Zapisano wyniki do pliku CSV: wyniki.csv


Wnioski 2:
- Adaboost ma lepsze wyniki, gdyż tworzy wiele drzew, ma duże korygowanie wag dla trudniejszych zbiorów, oraz ma większą odporność na przeuczenie. Kosztem lepszej jakości jest długi czas wykonania ze względu na złożoność klasyfikatora.
- Zwykłe drzewo jest proste i podatne na przeuczenie. Ale jest szybkie.
- RandomForest bez modyfikacji ma lepsze wyniki od zwykłego drzewa w każdym zbiorze z tego powodu, że tworzy wiele drzew i jest bardziej odporne na przeuczenie.
- Użyto paru hybryd oversamplingu/undersamplingu. SMOTETomek jako hybryda wypada nawet dobrze. SMOTEENN jest gorszy od samego undersamplingu. Ogólnie na tych zbiorach hybrydowe zbiory mają problemy.
- Użyto HistGradientBoostingClassifier. Nawet przy małych parametrach działa najwolniej i daje najlepsze wyniki
- Dodawanie undersamplerów innych niż RandomUnderSampler do SMOTE i SVMSMOTE doprowadzało do niedoboru sąsiadów dla klas pomimo małych parametrów
- BorderlineSMOTE jako jedyny może działać z undersamplerami na tych zbiorach, choć czasami 1 z X fitów nie powodzi się (nie zatrzymuje się jednak)


- Po pełnym wykonaniu pętli, dla HistGradientBoostingClassifier tylko hybryda SMOTETomek daje delikatnie lepsze wyniki. Poza tym głównie tylko oversamplery podwyższają wynik, a kombinacja zwykle zmniejsza.

- undersampler ClusterCentroids ma bardzo dobre wyniki dla zbioru SSH, i średnie w pozostałych zbiorach oprócz ALL, gdzie osiąga prawie najgorsze wyniki.



Wnioski 3:
- RFE absolutnie nie nadaje się do tego zbioru, gdyż ma duży koszt obliczeniowy i czasowy, gdzie nawet ze zwykłym DecisionTree klasyfikacja 6 modeli z 11 sekund wzrosła do 3-4 minut z nieznacznymi poprawkami
- Użyto SelectFromModel dla dużo szybszej selekcji cech kosztem jakości
- Do selekcji cech użyto estymatora SVC. Wyniki z selekcji cech są trochę losowe. Dla tych zbiorów bardzo niska wartość C poprawia jakość i stabilność wyników.
- Czas klasyfikacji wszystkich kombinacji zbiorów spadł z 50-75 minut do 15-20, głównie dlatego, że modele mają tylko 10% wybranych cech 